In [1]:
import tkinter as tk
from tkinter import messagebox
from datetime import datetime

In [2]:
# BankAccount Class (Encapsulation)
class BankAccount:
    def __init__(self, account_number, account_holder, balance=0):
        self.account_number = account_number
        self.account_holder = account_holder
        self.balance = balance
        self.transaction_history = []

    def deposit(self, amount):
        if amount > 0:
            self.balance += amount
            self.transaction_history.append(f"Deposited: {amount}")
            return True
        return False

    def withdraw(self, amount):
        if amount > 0 and self.balance >= amount:
            self.balance -= amount
            self.transaction_history.append(f"Withdrew: {amount}")
            return True
        return False

    def view_balance(self):
        return self.balance

    def view_transactions(self):
        return self.transaction_history

In [3]:
# SavingsAccount Class (Inheritance)
class SavingsAccount(BankAccount):
    def __init__(self, account_number, account_holder, balance=0, interest_rate=0.02):
        super().__init__(account_number, account_holder, balance)
        self.interest_rate = interest_rate

    def calculate_interest(self, months):
        return self.balance * (1 + self.interest_rate) ** months - self.balance

    def generate_statement(self):
        return (
            f"Savings Account Statement\n"
            f"Account Holder: {self.account_holder}\n"
            f"Balance: {self.balance}\n"
            f"Interest Rate: {self.interest_rate * 100}%"
        )



In [4]:
# CurrentAccount Class (Inheritance)
class CurrentAccount(BankAccount):
    def __init__(self, account_number, account_holder, balance=0, overdraft_limit=1000):
        super().__init__(account_number, account_holder, balance)
        self.overdraft_limit = overdraft_limit
        self.overdraft_used = 0

    def withdraw(self, amount):
        if amount > 0:
            total_available = self.balance + (self.overdraft_limit - self.overdraft_used)
            if total_available >= amount:
                if self.balance >= amount:
                    self.balance -= amount
                else:
                    overdraft_needed = amount - self.balance
                    self.overdraft_used += overdraft_needed
                    self.balance = 0
                self.transaction_history.append(f"Withdrew: {amount}")
                return True
        return False

    def view_balance(self):
        effective_balance = self.balance - self.overdraft_used
        return f"{effective_balance} (Overdraft Used: {self.overdraft_used})"

    def generate_statement(self):
        return (
            f"Current Account Statement\n"
            f"Account Holder: {self.account_holder}\n"
            f"Balance: {self.balance}\n"
            f"Overdraft Used: {self.overdraft_used}\n"
            f"Available Overdraft: {self.overdraft_limit - self.overdraft_used}"
        )


In [5]:
# Banking System GUI
class BankingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Banking Transaction System")
        self.accounts = {}  # Stores all accounts

        # Create GUI Components
        self.create_widgets()

    def create_widgets(self):
        # Labels
        tk.Label(self.root, text="Account Number:").grid(row=0, column=0, padx=10, pady=10)
        tk.Label(self.root, text="Account Holder:").grid(row=1, column=0, padx=10, pady=10)
        tk.Label(self.root, text="Initial Balance:").grid(row=2, column=0, padx=10, pady=10)
        tk.Label(self.root, text="Account Type:").grid(row=3, column=0, padx=10, pady=10)

        # Entry Fields
        self.account_number_entry = tk.Entry(self.root)
        self.account_number_entry.grid(row=0, column=1, padx=10, pady=10)
        self.account_holder_entry = tk.Entry(self.root)
        self.account_holder_entry.grid(row=1, column=1, padx=10, pady=10)
        self.balance_entry = tk.Entry(self.root)
        self.balance_entry.grid(row=2, column=1, padx=10, pady=10)

        # Account Type Dropdown
        self.account_type_var = tk.StringVar(value="Savings")
        tk.OptionMenu(self.root, self.account_type_var, "Savings", "Current").grid(row=3, column=1, padx=10, pady=10)

        # Buttons
        tk.Button(self.root, text="Create Account", command=self.create_account).grid(row=4, column=0, columnspan=2, pady=10)
        tk.Button(self.root, text="Deposit", command=self.deposit).grid(row=5, column=0, padx=10, pady=10)
        tk.Button(self.root, text="Withdraw", command=self.withdraw).grid(row=5, column=1, padx=10, pady=10)
        tk.Button(self.root, text="Transfer", command=self.transfer).grid(row=6, column=0, padx=10, pady=10)
        tk.Button(self.root, text="View Transactions", command=self.view_transactions).grid(row=6, column=1, padx=10, pady=10)
        tk.Button(self.root, text="Generate Statement", command=self.generate_statement).grid(row=7, column=0, columnspan=2, pady=10)

        # Label to Display Current Balance
        self.balance_label = tk.Label(self.root, text="Current Balance: -", fg="blue")
        self.balance_label.grid(row=8, column=0, columnspan=2, pady=10)

    def clear_entries(self):
        """Clear all input fields."""
        self.account_number_entry.delete(0, tk.END)
        self.account_holder_entry.delete(0, tk.END)
        self.balance_entry.delete(0, tk.END)

    def create_account(self):
        account_number = self.account_number_entry.get()
        account_holder = self.account_holder_entry.get()
        balance = float(self.balance_entry.get()) if self.balance_entry.get() else 0
        account_type = self.account_type_var.get()

        if account_number in self.accounts:
            messagebox.showerror("Error", "Account number already exists!")
            return

        if account_type == "Savings":
            account = SavingsAccount(account_number, account_holder, balance)
        else:
            account = CurrentAccount(account_number, account_holder, balance)

        self.accounts[account_number] = account
        messagebox.showinfo("Success", f"{account_type} Account created successfully!")
        self.clear_entries()  # Clear input fields after account creation

        # Update the balance label
        self.balance_label.config(text=f"Current Balance: {account.view_balance()}")

    def deposit(self):
        account_number = self.account_number_entry.get()
        amount = float(self.balance_entry.get()) if self.balance_entry.get() else 0

        if account_number in self.accounts:
            if self.accounts[account_number].deposit(amount):
                messagebox.showinfo("Success", f"Deposited {amount} successfully!")
                self.balance_label.config(text=f"Current Balance: {self.accounts[account_number].view_balance()}")
            else:
                messagebox.showerror("Error", "Invalid deposit amount!")
        else:
            messagebox.showerror("Error", "Account not found!")

    def withdraw(self):
        account_number = self.account_number_entry.get()
        amount = float(self.balance_entry.get()) if self.balance_entry.get() else 0

        if account_number in self.accounts:
            if self.accounts[account_number].withdraw(amount):
                messagebox.showinfo("Success", f"Withdrew {amount} successfully!")
                self.balance_label.config(text=f"Current Balance: {self.accounts[account_number].view_balance()}")
            else:
                messagebox.showerror("Error", "Insufficient balance or invalid amount!")
        else:
            messagebox.showerror("Error", "Account not found!")

    def transfer(self):
        from_account = self.account_number_entry.get()
        to_account = self.account_holder_entry.get()  # Reusing this field for destination account
        amount = float(self.balance_entry.get()) if self.balance_entry.get() else 0

        if from_account in self.accounts and to_account in self.accounts:
            if self.accounts[from_account].withdraw(amount):
                self.accounts[to_account].deposit(amount)
                messagebox.showinfo("Success", f"Transferred {amount} successfully!")
                self.balance_label.config(text=f"Current Balance: {self.accounts[from_account].view_balance()}")
            else:
                messagebox.showerror("Error", "Transfer failed! Check balance or amount.")
        else:
            messagebox.showerror("Error", "One or both accounts not found!")

    def view_transactions(self):
        account_number = self.account_number_entry.get()
        if account_number in self.accounts:
            transactions = self.accounts[account_number].view_transactions()
            messagebox.showinfo("Transaction History", "\n".join(transactions))
        else:
            messagebox.showerror("Error", "Account not found!")

    def generate_statement(self):
        account_number = self.account_number_entry.get()
        if account_number in self.accounts:
            statement = self.accounts[account_number].generate_statement()
            messagebox.showinfo("Account Statement", statement)
        else:
            messagebox.showerror("Error", "Account not found!")


In [6]:
# Main Application
if __name__ == "__main__":
    root = tk.Tk()
    app = BankingApp(root)
    root.mainloop()

KeyboardInterrupt: 